In [ ]:
# --quiet suprime o output de texto, para manter o notebook mais legível
!pip install --quiet 'crewai==0.28.8'
!pip install --quiet 'crewai_tools==0.1.6'
!pip install --quiet 'langchain-community'
!pip install --quiet 'duckduckgo-search>=6.1.7'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 8.9 MB/s eta 0

In [ ]:
from crewai_tools import FileReadTool, ScrapeWebsiteTool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.llms import OpenAI

In [ ]:
# Cria a ferramenta a ser usada para buscar na internet
search_tool = DuckDuckGoSearchRun()
# Cria a ferramenta a ser usada para obter mais informações sobre um site em específico
scrape_tool = ScrapeWebsiteTool()

In [ ]:
from crewai import Agent, Task, Crew
import os

os.environ['OPENAI_API_KEY'] = ''

# Cria um agente que vai buscar e reunir informações empresariais sobre a empresa a ser coberta.
# Define seu objetivo e backstory tal que possa atender à tarefa. O mesmo vai também ser feito para todos os outros agentes.
company_research_agent = Agent(
    role="Company Research Agent",
    goal="Gather detailed information about the target company, including their business, products, services, and market position.",
    backstory=(
        "Tasked with collecting comprehensive data about potential clients to inform the sales pitch."
    ),
    allow_delegation=True,
    verbose=True
)

# Define a tarefa esperada do agente, incluindo detalhamento sobre qual o output esperado.
# O mesmo vai também ser feito para todos os outros agentes.
company_research_task = Task(
    description=(
        "Collect comprehensive data about {company_name} from their website ({company_website}) and other sources. "
        "Include details about their business, products, services, and market position."
    ),
    expected_output=(
        "Detailed report on {company_name}'s business, products, services, and market position."
    ),
    agent=company_research_agent,
    tools=[search_tool, scrape_tool]
)

In [ ]:
# Cria um agente que vai analisar com detalhe o website da empresa, buscand por melhoras possíveis.
website_analysis_agent = Agent(
    role="Website Analysis Agent",
    goal="Analyze the target company's website for areas of improvement, including design, performance, and user experience.",
    backstory=(
        "Focuses on identifying specific issues and opportunities for enhancement on the client's website."
    ),
    allow_delegation=True,
    verbose=True
)

# Define a tarefa esperada do agente, incluindo detalhamento sobre qual o output esperado.
website_analysis_task = Task(
    description=(
        "Analyze {company_name}'s website ({company_website}) to identify areas of improvement in design, performance, and user experience."
    ),
    expected_output=(
        "Detailed analysis report highlighting areas of improvement for {company_name}'s website, including design, performance, and user experience."
    ),
    agent=website_analysis_agent,
    tools=[scrape_tool, search_tool]
)

In [ ]:
# Cria um agente que vai analisar o conteúdo do site, de um ponto de vista de marketing, buscando formas de melhorar engajamento e conversão.
content_strategy_agent = Agent(
    role="Content Strategy Agent",
    goal="Develop content strategies and identify how improved content can drive engagement and conversions.",
    backstory=(
        "Specializes in creating a compelling narrative and strategy for content improvements on the client's website."
    ),
    allow_delegation=True,
    verbose=True
)

# Define a tarefa esperada do agente, incluindo detalhamento sobre qual o output esperado
content_strategy_task = Task(
    description=(
        "Develop a content strategy for {company_name}'s website ({company_website}) to enhance user engagement and conversion rates."
    ),
    expected_output=(
        "Comprehensive content strategy for {company_name}'s website, including recommendations for content improvements."
    ),
    agent=content_strategy_agent,
    tools=[search_tool, scrape_tool]
)

In [ ]:
# Cria um agente que vai gerar um sales pitch ("discurso de vendas") usando as pesquisas dos outros agentes.
# Tem o intuito de convencer a companhia alvo que seu website precisa ser modificado com a nossa companhia "Code With Prince",
#  com base no que foi encontrado pelos demais agentes.
sales_pitch_agent = Agent(
    role="Sales Pitch Agent",
    goal="Create a personalized and persuasive sales pitch to convince the target company to have their website modified by 'Code With Prince'.",
    backstory=(
        "Combines research, analysis, and content strategy to craft a compelling sales pitch."
    ),
    allow_delegation=False,
    verbose=True
)

# Define a tarefa esperada do agente, incluindo detalhamento sobre qual o output esperado
sales_pitch_task = Task(
    description=(
        "Combine the research, analysis, and content strategy to create a personalized and persuasive sales pitch for {company_name} "
        "to convince them to have their website modified by 'Code With Prince'."
    ),
    expected_output=(
        "Customized sales pitch for {company_name}, highlighting the benefits of having their website modified by 'Code With Prince'."
    ),
    agent=sales_pitch_agent,
    tools=[search_tool, scrape_tool]
)

In [ ]:
# Inicializa a Crew (representa o grupo de agente, quais as tasks deles e como devem colaborar).
crew = Crew(
    agents=[
        company_research_agent,
        website_analysis_agent,
        content_strategy_agent,
        sales_pitch_agent
    ],
    tasks=[
        company_research_task,
        website_analysis_task,
        content_strategy_task,
        sales_pitch_task
    ],
    verbose=2,
    memory=True,
    cache=True
)

In [ ]:
# Define a companhia alvo a ser analisada, e o seu site.
inputs = {
    "company_name": "CrewAI",
    "company_website": "https://crewai.com"
}

# Começa o trabalho da Crew.
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Company Research Agent
 [INFO]: == Starting Task: Collect comprehensive data about CrewAI from their website (https://crewai.com) and other sources. Include details about their business, products, services, and market position.


> Entering new CrewAgentExecutor chain...
The first step will be to examine the company's website to understand their business, products, and services. I should use the tool "Read website content" to gather this information.

Action: 
Read website content

Action Input: 
{"website_url": "https://crewai.com"}
 

crewAI - Platform for Multi AI Agents Systems
×
 Documentation
 Chat with our docs
 Join our Discord
 Blog
Menu
×
Documentation
Chat with our docs
Join our Discord
Blog
Login
Sign Up
														AI Agents forreal use cases
														Most AI agent frameworks are hard to use.
														We provide power with simplicity.
														Automate your most important workflows quickly.
1,400,000+
Multi-Agent Crews run last 

In [ ]:
# Usa Markdown para mostrar a resposta gerada.
from IPython.display import Markdown

# Armazena o resultado em um arquivo.
with open("result.md", "w") as file:
    file.write(result)

Markdown(result)

Dear CrewAI team,

I hope this message finds you well. I've had the opportunity to explore your website, and I must commend you on the fantastic work you're doing in the field of AI and automation.

However, after a thorough analysis, we believe there are some areas where your website could benefit from a few enhancements. That's where we, 'Code With Prince', come into the picture. We are passionate about helping businesses like yours optimize their online presence, and here's how we propose to do it:

1. **User Experience**: We noticed that while your site is rich in information, it could use a more streamlined layout and navigation. Our team will work on categorizing your menu items effectively, removing any unnecessary clutter, and ensuring the website follows a logical path, making it more user-friendly.

2. **Content Clarity**: We aim to make your platform's descriptions and features more concise and accessible. We'll minimize the use of technical jargon and incorporate more visuals to illustrate complex concepts.

3. **Engaging Content**: We'll help incorporate customer testimonials and case studies into your website. Adding a blog section offering insights and updates on AI and automation will keep visitors engaged and establish CrewAI as a thought leader in the field.

4. **Calls to Action**: We'll revamp your calls to action to be more persuasive, clearly stating what action the visitor is expected to take and the benefits they will receive.

5. **Conversion Rate Optimization**: We'll help you offer a free trial or demo of the platform to attract potential customers. This strategy reduces the perceived risk and encourages more sign-ups.

6. **Performance**: We'll ensure your website meets the Core Web Vitals standards, including Largest Contentful Paint (LCP), Interaction to Next Paint (INP), and Cumulative Layout Shift (CLS).

7. **Mobile Responsiveness**: We will optimize your website for mobile responsiveness, ensuring a good user experience on all devices.

8. **Security**: We'll highlight the security measures in place to protect user data, building trust with potential customers.

Our goal is to make your website not only visually appealing but also highly functional and effective in driving conversions. We're excited about the possibility of working with you and look forward to hearing from you soon.

Best Regards,

[Your Name]
Sales Agent, Code With Prince